In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
import plotly.graph_objects as go

import pandas_ta 


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 

from scripts.indicators import resample, join_after_resample, HeikinAshi,zigzag

from binance.client import Client

#Conexion con Binance sin API-KEY
client = Client()

c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest


In [2]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [8]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_ETHUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(60*4))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2023-02-17'
	end = dt.datetime.now().strftime("%Y-%m-%d") #'2025-02-17'  
	print(end)
	symbol = 'BTCUSDT'
	interval = '1d'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

In [9]:

#Fuente
#https://www.youtube.com/watch?v=IFLTmZBjEWU

def calculate_optimal_rsi(df, rsi_min_length=4, rsi_count=17, ma_length=14, optimal_length=200, backup_length=14, optimal_type="All Crossings"):
    cross_percents = []
    df['RSI'] = None
    for i in range(rsi_count):
        rsi_length = i + rsi_min_length
        df[f'RSI_{rsi_length}'] = pandas_ta.rsi(df['close'],length=rsi_length)
        df[f'RSI_MA_{rsi_length}'] = df[f'RSI_{rsi_length}'].rolling(ma_length).mean()
        
        cross_percent = 0
        cross_count = 0
        cross_type = 0
        cross_close = 0
        in_extremity = False
        
        for j in range(optimal_length):
            if j >= len(df):
                break
            
            current_rsi = df[f'RSI_{rsi_length}'].iloc[-j]
            current_ma = df[f'RSI_MA_{rsi_length}'].iloc[-j]
            
            cross_over = current_rsi > current_ma
            cross_under = current_rsi < current_ma
            
            if optimal_type == "All Crossings":
                current_over = cross_over
                current_under = cross_under
            else:
                current_over = cross_over and current_rsi <= 40
                current_under = cross_under and current_rsi >= 60
            
            if current_over:
                if cross_type != 0:
                    cross_percent += cross_close / df['close'].iloc[-j]
                    cross_count += 1
                cross_close = df['close'].iloc[-j]
                cross_type = 1
                in_extremity = not in_extremity
            elif current_under:
                if cross_type != 0:
                    cross_percent += df['close'].iloc[-j] / cross_close
                    cross_count += 1
                cross_close = df['close'].iloc[-j]
                cross_type = -1
                in_extremity = not in_extremity
        
        avg_percent = cross_percent / cross_count if cross_count > 0 else -100000
        cross_percents.append((rsi_length, avg_percent))
    
    best_rsi = max(cross_percents, key=lambda x: x[1])[0] if cross_percents else backup_length
    return best_rsi

def apply_strategy(df, rsi_length, ma_length=14):
    df['Optimal_RSI'] = pandas_ta.rsi(df['close'],length=rsi_length)
    df['MA'] = df['close'].rolling(window=200).mean()
    bb_length = 20
    std_outer = 2.0
    std_inner = 1.6185
    rsi_bb = pandas_ta.bbands(df['Optimal_RSI'],length=bb_length, std=2.0)
    #df['RSI_MA'] = df['Optimal_RSI'].ewm(span=14).mean()
    df['RSI_MA'] = df['Optimal_RSI'].rolling(window=14).mean()
    df['RSI_BBU'] = rsi_bb[f'BBU_{bb_length}_2.0']
    df['RSI_BBL'] = rsi_bb[f'BBL_{bb_length}_2.0']
    
    #df['signal'] = np.where((df['Optimal_RSI'] < 100) & (df['Optimal_RSI'] > df['RSI_BBU']) & (df['Optimal_RSI'].shift(1) <= df['RSI_BBU'].shift(1)),1,0)
    #df['signal'] = np.where((df['Optimal_RSI'] > 0) & (df['Optimal_RSI'] < df['RSI_BBL']) & (df['Optimal_RSI'].shift(1) >= df['RSI_BBL'].shift(1)),-1,df['signal'])
    df['signal'] = np.where((df['close'] > df['MA']) & (df['Optimal_RSI'] < 50) & (df['Optimal_RSI'] > df['RSI_MA']) & (df['Optimal_RSI'].shift(1) <= df['RSI_MA'].shift(1)),1,0)
    df['signal'] = np.where( (df['Optimal_RSI'] > 50) & (df['Optimal_RSI'] < df['RSI_MA']) & (df['Optimal_RSI'].shift(1) >= df['RSI_MA'].shift(1)),-1,df['signal'])
    
	#Stop-loss sin close baja de la MA
    df['signal'] = np.where((df['close'] < df['MA']) & (df['close'].shift(1) > df['MA'].shift(1)),-1,df['signal']) 
    return df

def calcular_profit(df):

    cash = 100000
    stock = 0
    qty = 0
    df['pnl'] = 0
    df['result'] = 0
    df['trade'] = 0
    
    open_i = 0
    open_pos_cash = 0
    for i,row in df.iterrows():
        if row['signal']>0 and cash>0:
            open_pos_cash = cash
            open_i = i
            qty = cash/row['close']
            stock += qty
            cash = 0
        elif row['signal']<0 and qty>0:
            cash = qty*row['close']
            stock = 0
            qty = 0
    
            result = round(((cash/df.loc[open_i]['pnl'])-1)*100,1)
            df.loc[i,'trade'] = result
            df.loc[open_i,'trade'] = result
    
            open_pos_cash = 0
            open_i = 0
            
        df.loc[i,'pnl'] = cash + stock*row['close']
    
    
    #Quitando las señales que no alcanzan el profit
    print(df[df['signal']>0]['trade'].describe())
    

In [10]:
# Ejemplo de uso
rsi_length = calculate_optimal_rsi(df)
df = apply_strategy(df, rsi_length)
calcular_profit(df)

df['buy']  = np.where( df['signal']==1  , df['close'] , None )
df['sell'] = np.where( df['signal']==-1 , df['close'] , None )


count    76.000000
mean      0.357895
std       5.920772
min     -21.300000
25%      -1.050000
50%       0.000000
75%       1.425000
max      16.400000
Name: trade, dtype: float64


In [11]:
to_show = df.copy()#[200:600]
if 'volume' in to_show.columns:
	to_show.drop(columns=['volume'],inplace=True)
indicators = [
     {'col': 'MA','color': 'gray','row': 1, 'mode':'lines',},
     #{'col': 'ZigZag','name': 'ZigZag','color': 'yellow','row': 1, 'mode':'markers',},

        ]
indicators_out = [
     {'col': 'Optimal_RSI','color': 'yellow','row': 1, 'mode':'lines',},
     {'col': 'RSI_MA','color': 'green','row': 1, 'mode':'lines',},
     #{'col': 'RSI_BBU','color': 'green','row': 1, 'mode':'lines',},
     #{'col': 'RSI_BBL','color': 'green','row': 1, 'mode':'lines',},
       ]
#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        {'df':to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
        {'df':to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
     ]

fig = fn.ohlc_chart(to_show,indicators=indicators,indicators_out = indicators_out, events=events) #
fig.show()

In [12]:
def optimal_rsi(df, min_length=7, max_length=21, step=1):
    optimal_rsi_values = []
    
    for i in range(0, len(df), 100):  # Procesa en bloques de 100 velas
        df_subset = df.iloc[i:i+100].copy()
        best_rsi = None
        best_sharpe = -np.inf
        
        for rsi_length in range(min_length, max_length + 1, step):
            df_subset[f'RSI_{rsi_length}'] = pandas_ta.rsi(df_subset['close'], length=rsi_length)
            
            # Cálculo del Sharpe Ratio simple para optimización
            rsi_returns = df_subset[f'RSI_{rsi_length}'].pct_change().dropna()
            sharpe_ratio = rsi_returns.mean() / rsi_returns.std() if rsi_returns.std() != 0 else 0
            
            if sharpe_ratio > best_sharpe:
                best_sharpe = sharpe_ratio
                best_rsi = rsi_length
        
        optimal_rsi_values.append((i, best_rsi))
    
    return optimal_rsi_values

# Ejemplo de uso (el DataFrame df ya debe estar creado)
optimal_rsi_values = optimal_rsi(df)
print(optimal_rsi_values)

[(0, 7), (100, 20), (200, 7), (300, 7), (400, 7), (500, 7), (600, 7), (700, 16), (800, 8), (900, 7), (1000, 7), (1100, 21), (1200, 12), (1300, 21), (1400, 21), (1500, 7), (1600, 9), (1700, 8), (1800, 7), (1900, 15), (2000, 7), (2100, 15), (2200, 8), (2300, 17), (2400, 20), (2500, 14), (2600, 7), (2700, 7), (2800, 7), (2900, 7), (3000, 9), (3100, 11), (3200, 10), (3300, 8), (3400, 7), (3500, 7), (3600, 7), (3700, 7), (3800, 7), (3900, 7), (4000, 7), (4100, 7), (4200, 12), (4300, 7), (4400, 19), (4500, 7), (4600, 7), (4700, 7), (4800, 7), (4900, 7), (5000, 16), (5100, 7), (5200, 7), (5300, 9), (5400, 7), (5500, 7), (5600, 19), (5700, 7), (5800, 7), (5900, 7), (6000, 16), (6100, 8), (6200, 7), (6300, 7), (6400, 7), (6500, 7), (6600, 9), (6700, 13), (6800, 8), (6900, 7)]
